In [1]:
from scipy.io import loadmat
from scipy.sparse import find
from scipy.sparse.csgraph import dijkstra
from scipy.linalg import sqrtm
from scipy.cluster.hierarchy import linkage, dendrogram, cut_tree, to_tree, ClusterNode
from scipy.spatial.distance import pdist, squareform
from scipy.stats import wasserstein_distance, entropy
from sklearn.metrics import adjusted_rand_score
import numpy as np
import numpy.linalg as lalg
import matplotlib.pyplot as plt
import networkx as nx
from networkx.generators.community import LFR_benchmark_graph
from pickle import dump, load, HIGHEST_PROTOCOL
%matplotlib inline

# wrangle data

In [2]:
def largest_connected_component(G):
    G = G.subgraph(max(nx.connected_components(G), key=len))
    G = nx.relabel.convert_node_labels_to_integers(G, label_attribute='orig')
    return G

In [3]:
def wrangle_karate():
    mat = loadmat('karate.mat')
    S = mat['Problem'][0][0][2]
    G = nx.Graph(S)

    mr_hi = set([1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 17, 18, 20, 22])
    split = {}
    cols = {}
    for i in G.nodes:
        split[i] = 0 if i+1 in mr_hi else 1
        cols[i] = (1,0,0,1) if i+1 in mr_hi else (0,0,0,1)
    nx.set_node_attributes(G, split, 'cluster')
    nx.set_node_attributes(G, cols, 'color')
    G.graph['name'] = 'karate'
    return largest_connected_component(G)

In [4]:
def wrangle_football():
    # data fixed by Tim Evans
    mat = loadmat('football_fixed.mat')
    I,J = mat['links']
    G = nx.Graph()
    G.add_edges_from(zip(I,J))

    confs = {}
    cols = {}
    cmap = plt.get_cmap('tab20')
    for node in mat['nodes']:
        idx = int(node[0])
        conf = int(node[2])
        assert idx not in confs.keys()
        if conf>10:
            confs[idx] = conf
            cols[idx] = (0,0,0,1)
        else:
            confs[idx] = conf
            cols[idx] = cmap(conf/11)
    nx.set_node_attributes(G, confs, 'cluster')
    nx.set_node_attributes(G, cols, 'color')
    G.graph['name'] = 'football'
    return largest_connected_component(G)

In [5]:
def wrangle_caltech():
    mat = loadmat('Caltech36.mat')
    G = nx.Graph(mat['A'])

    houses = {}
    cols = {}
    cmap = plt.get_cmap('tab10')
    for i,x in enumerate(mat['local_info']):
        house = x[4]
        if house==0:
            houses[i] = house
            cols[i] = (0,0,0,1)
        else:
            houses[i] = house
            cols[i] = cmap((house-165)/10)
    nx.set_node_attributes(G, houses, 'cluster')
    nx.set_node_attributes(G, cols, 'color')
    G.graph['name'] = 'caltech'
    return largest_connected_component(G)

In [6]:
def wrangle_lesmis():
    G = largest_connected_component(nx.les_miserables_graph())
    # no ground truth
    cmap = plt.get_cmap('tab10')
    nx.set_node_attributes(G, {i:0 for i in G.nodes}, 'cluster')
    nx.set_node_attributes(G, {i:cmap(0) for i in G.nodes}, 'color')
    return G

In [7]:
def generate_lfr():
    G = LFR_benchmark_graph(n=250, tau1=3, tau2=1.5, mu=.1, average_degree=5, min_community=20, seed=0)
    communities = nx.get_node_attributes(G, 'community')
    seen = set()
    clust_idx = 0
    clusts = {}
    cols = {}
    cmap = plt.get_cmap('tab10')
    for idx,community in communities.items():
        if idx not in seen:
            for member in community:
                clusts[member] = clust_idx
                cols[member] = cmap(clust_idx/10)
                seen.add(member)
            clust_idx += 1
    nx.set_node_attributes(G, clusts, 'cluster')
    nx.set_node_attributes(G, cols, 'color')
    G.remove_edges_from(nx.selfloop_edges(G))
    G.graph['name'] = 'lfr'
    return largest_connected_component(G)

# cluster data

In [8]:
def is_metric(dist, eps=0):
    square = squareform(dist)
    n,m = square.shape
    assert n==m
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if square[i,j] > square[i,k]+square[k,j]+eps:
                    print(f'{i} {j} {k}')
                    print(square[i,j])
                    print(square[i,k]+square[k,j])
                    return False
    return True

def draw_dendrogram(G, hierarchy, figsize=(20,5)):
    plt.figure(figsize=figsize)
    plt.rcParams["font.weight"] = "bold"
    dend = dendrogram(hierarchy, leaf_rotation=90, leaf_font_size=8)

    ticks, labels = plt.xticks()
    for label in labels:
        idx = int(label.get_text())
        label.set_color(G.nodes[idx]['color'])
    plt.xticks(ticks, labels)

def eval_ARI(G, hierarchy):
    """calculates the adjusted rand index to evaluate cluster quality"""
    if hierarchy is None:
        return (None, 'infinity') # might happen if there are infinite distances so KL is undefined
    try:
        cut = cut_tree(hierarchy).transpose()
    except ValueError:
        return (None, 'non-metric') # might happen if there are negative distances in tree due to non-metric
    
    attrs = nx.get_node_attributes(G, 'cluster')
    n = len(G)
    gtruth = []
    ignore = set() # unuseful labels are set to -1 earlier
    for idx,label in attrs.items():
        if label >= 0:
            gtruth.append(label)
        else:
            ignore.add(idx)
    labelset = set(gtruth)
    predicted = []
    
    # try cutting at every dendrogram branch
    best = -float('inf')
    argbest = -1
    for n_clust, clusters in enumerate(cut):
        score = adjusted_rand_score(gtruth, clusters)
        if score > best:
            best = score
            argbest = n_clust
    return best, n-argbest

In [9]:
# functions to create high dimensional embeddings based on markov chains

def embed_snapshot_markov(G, steps, degrees):
    '''embedding of pons & lapaty, including extra weighting using degrees'''
    # get markov matrix P
    n = len(G)
    A = nx.to_numpy_array(G)
    D = np.identity(n) * np.sum(A, axis=1)
    P = lalg.inv(D) @ A
    
    # steps
    P = lalg.matrix_power(P, steps)
    if degrees:
        P = lalg.inv(sqrtm(D)) @ P
    return P

def embed_damped_markov(G, damping, steps=100):
    '''weights each markov chain state according to a damping factor'''
    # get markov matrix P
    n = len(G)
    A = nx.to_numpy_array(G)
    D = np.identity(n) * np.sum(A, axis=1)
    P = lalg.inv(D) @ A
    
    # get weighted sum of all snapshots
    state = np.array(P)
    final = np.array(P)
    totaldamp = 1
    currdamp = damping
    for i in range(1,steps):
        state = state @ P
        final += currdamp * state
        totaldamp += currdamp
        currdamp *= damping
    final /= totaldamp
    return final

In [10]:
# linkage methods

def cluster_walktrap(embedding, method='ward', opt_ord=False):
    dist = pdist(embedding, metric='euclidean')
    # print(is_metric(dist))
    return linkage(dist, method=method, optimal_ordering=opt_ord)
    
def KL_symmetric(embedding, method='ward', opt_ord=False):
    if min(np.nditer(embedding)) <= 0:
        return None
    dist = pdist(embedding, metric=lambda x,y: entropy(x,y)+entropy(y,x))
    # print(is_metric(dist))
    return linkage(dist, method=method, optimal_ordering=opt_ord)

def wasserstein(embedding, method='ward', opt_ord=False):
    dist = pdist(embedding, wasserstein_distance)
    # print(is_metric(squareform(dist), 1e-10))
    return linkage(dist, method=method, optimal_ordering=opt_ord)

def bacon(G, method='ward', opt_ord=False):
    A = nx.to_numpy_array(G)
    bacon = dijkstra(A, directed=False, unweighted=True)
    dist = squareform(bacon)

    # print(is_metric(dist))
    return linkage(dist, method=method, optimal_ordering=opt_ord)

def jaccard(G, method='ward', opt_ord=False):
    A = nx.to_numpy_array(G)
    dist = pdist(A, metric='jaccard')

    # print(is_metric(dist))
    return linkage(dist, method=method, optimal_ordering=opt_ord)

def bacon_jaccard(G, method='ward', opt_ord=False):
    A = nx.to_numpy_array(G)
    jacc = pdist(A, metric='jaccard')
    bacon = dijkstra(A, directed=False, unweighted=True)
    dist = jacc * squareform(bacon)

    # print(is_metric(dist))
    return linkage(dist, method=method, optimal_ordering=opt_ord)

def commute(G, method='ward', opt_ord=False):
    # first get unnormalised laplacian
    n = len(G)
    A = nx.to_numpy_array(G)
    D = np.identity(n) * np.sum(A, axis=1)
    L = D-A
    
    # get moore-penrose pseudo-inverse
    Lplus = lalg.pinv(L)
    vol = 2*len(G.edges)
    
    # use to calculate ECT
    def commute(i,j):
        return vol * (Lplus[i,i] + Lplus[j,j] - 2*Lplus[i,j])
    
    dist = []
    for i in range(n):
        for j in range(i+1,n):
            dist.append(commute(i,j))
            
#     print(is_metric(dist, 1e-5))
    return linkage(dist, method=method, optimal_ordering=opt_ord)

In [11]:
# convert 

def to_linkage(root):
    nbranches = 0
    nleaves = 0
    def count(node):
        assert node is not None
        if node.left is None and node.right is None:
            nonlocal nleaves
            nleaves += 1
            return
        nonlocal nbranches
        nbranches += 1
        count(node.left)
        count(node.right)
    
    count(root)
    linkage = np.zeros((nbranches,4))
    
    def link(node):
        assert node is not None
        if node.left is None and node.right is None:
            return
        linkage[node.id-nleaves,0] = node.left.id
        linkage[node.id-nleaves,1] = node.right.id
        linkage[node.id-nleaves,2] = node.dist
        linkage[node.id-nleaves,3] = node.count
        link(node.left)
        link(node.right)
        
    link(root)
    return linkage

In [12]:
# divisive clustering of girvan and newman

def girvan_newman(G):
    G = G.copy() # required because this function destroys the graph
    n = nx.number_of_nodes(G)
    m = nx.number_of_edges(G)
    
    clust_idx = 2*n - 2
    root = ClusterNode(clust_idx, dist=m, count=n)
    clusters = {}
    for i in range(n):
        clusters[i] = root
    
    ncc_old = 1
    for k in range(m):
        betweenness = nx.edge_betweenness_centrality(G)
        most_between = max(betweenness, key=lambda x: betweenness[x])
        i = most_between[0]
        j = most_between[1]
        G.remove_edge(i,j)
        
        ncc = nx.number_connected_components(G)
        if ncc > ncc_old:
            
            clust = clusters[i]
            clust_2 = clusters[j]
            if clust != clust_2:
                raise Exception("sumting wong")
                
            comp_1 = nx.node_connected_component(G, i)
            comp_2 = nx.node_connected_component(G, j)
            
            if len(comp_1) < len(comp_2):
                i,j = j,i
                comp_1,comp_2 = comp_2,comp_1
            
            if len(comp_1) == 1:
                clust.left = ClusterNode(i, dist=0, count=1)
            else:
                clust_idx -= 1
                clust.left = ClusterNode(clust_idx, dist=m-k-1, count=len(comp_1))
                
            if len(comp_2) == 1:
                clust.right = ClusterNode(j, dist=0, count=1)
            else:
                clust_idx -= 1
                clust.right = ClusterNode(clust_idx, dist=m-k-1, count=len(comp_2))
            
            for idx in comp_1:
                clusters[idx] = clust.left
            for idx in comp_2:
                clusters[idx] = clust.right
                
        ncc_old = ncc
            
    return to_linkage(root)

# do clustering

In [13]:
# big slow function to measure quality of every clustering method

def do_experiment(G, method):
    # jaccard * bacon
    results = {}
    results['bacon'] = eval_ARI(G, bacon(G, method, False))
    results['jacc'] = eval_ARI(G, jaccard(G, method, False))
    results['baconjacc'] = eval_ARI(G, bacon_jaccard(G, method, False))

    # euclidean commute time
    results['commute'] = eval_ARI(G, commute(G, method, False))

    # walktrap, KL, wasserstein (all with both types of embedding)
    all_steps = [1,2,3,4,5,6,7,8,9,10,100]
    for steps in all_steps:
        embedding = embed_snapshot_markov(G, steps, degrees=True)
        results[f'snapshotD_walktrap_{steps:03d}'] = eval_ARI(G, cluster_walktrap(embedding, method, False))
        results[f'snapshotD_KL_{steps:03d}'] = eval_ARI(G, KL_symmetric(embedding, method, False))
        results[f'snapshotD_wasserstein_{steps:03d}'] = eval_ARI(G, wasserstein(embedding, method, False))

        embedding = embed_snapshot_markov(G, steps, degrees=False)
        results[f'snapshotI_walktrap_{steps:03d}'] = eval_ARI(G, cluster_walktrap(embedding, method, False))
        results[f'snapshotI_KL_{steps:03d}'] = eval_ARI(G, KL_symmetric(embedding, method, False))
        results[f'snapshotI_wasserstein_{steps:03d}'] = eval_ARI(G, wasserstein(embedding, method, False))

    # damped markov embedding
    all_damps = [.1,.2,.3,.4,.5,.6,.7,.8,.85,.9,1]
    for damp in all_damps:
        embedding = embed_damped_markov(G, damp, steps=100)
        dampstr = str(damp).replace('.','p')
        results[f'damped_walktrap_{dampstr}'] = eval_ARI(G, cluster_walktrap(embedding, method, False))
        results[f'damped_KL_{dampstr}'] = eval_ARI(G, KL_symmetric(embedding, method, False))
        results[f'damped_wasserstein_{dampstr}'] = eval_ARI(G, wasserstein(embedding, method, False))

    with open(f'results/ARI_{method}_{G.graph["name"]}.pkl', 'wb') as f: 
        dump(results, f, HIGHEST_PROTOCOL)

def do_girvannewman(G):
    result = eval_ARI(G, girvan_newman(G))
    with open(f'results/ARI_girvannewman_{G.graph["name"]}.pkl', 'wb') as f: 
        dump(results, f, HIGHEST_PROTOCOL)

In [ ]:
graphs = [wrangle_karate(), wrangle_football(), wrangle_caltech(), generate_lfr()]
methods = ['single', 'complete', 'average', 'weighted', 'ward']

for graph in graphs:
    for method in methods:
        print(f'{graph.graph["name"]} {method}')
        do_experiment(graph, method)

for graph in graphs:
    print(f'{graph.graph["name"]} gn')
    do_girvannewman(G)

karate single
karate complete


# plot clustering

In [ ]:
# plot results of above

def plot_experiment(name, method, total=None):
    with open(f'results/ARI_{method}_{name}.pkl', 'rb') as f:
        results = load(f)

    fig, axes = plt.subplots(nrows=3, ncols=3)
    fig.suptitle(name, fontweight='bold')
    colmap = {"snapshotI":0, "snapshotD":1, "damped":2}
    rowmap = {"walktrap":0, "KL":1, "wasserstein":2}
    
    bestARI = max((res[0] if res[0] is not None else -1) for res in results.values())
    lines = {}
    for label, result in sorted(results.items()):
        
        if result[0] is None:
            continue # when KL does not converge
        if result[0] == bestARI:
            print(f'TODO HIGHLIGHT {label} {result[0]:.3f} {name}')
            
        split = label.split('_')
        if len(split)==3:
            ax_col = colmap[split[0]]
            ax_row = rowmap[split[1]]
            x = float(split[2].replace('p','.'))
            y = result[0]
            coord = (ax_row,ax_col)
            if coord not in lines.keys():
                lines[coord] = ([x], [y])
            else:
                lines[coord][0].append(x)
                lines[coord][1].append(y)
        else:
            print(f"{label:<25} {result[0]:.3f} {result[1]} {name}")
            # TODO: put this in another graph
        
        if total is not None:
            if label not in total.keys():
                total[label] = [result]
            else:
                total[label].append(result)

    # plot lines and remove irrelevant ticks
    for coord,line in lines.items():
        axis = axes[coord[0],coord[1]]
        axis.plot(line[0], line[1])
        axis.set_ylim(-.1,1.1)
        axis.grid(True)
        
        if coord[0] < 2:
            axis.set_xticklabels(['' for _ in axis.get_xticklabels()])
        if coord[1] > 0:
            axis.set_yticklabels(['' for _ in axis.get_yticklabels()])
        if coord[1] < 2:
            axis.set_xlim(0,10)
        else:
            axis.set_xlim(0,1)
        
    # label relevant axis
    for label,col in colmap.items():
        axes[0,col].title.set_text(label)
    for label,row in rowmap.items():
        axes[row,0].set_ylabel(label)
    
    fig.savefig(f'figures/ARI_{method}_{name}.png')
    
def plot_experiments(names, method):
    total_ARI = {}
    for name in names:
        plot_experiment(name, method, total_ARI)
    
    for label,results in total_ARI.items():
        n = len(results)
        if n==len(names):
            total_ARI[label] = (sum(res[0] for res in results) / n, sum(res[1] for res in results) / n)
        else:
            total_ARI[label] = (-1,0)
        with open(f'results/ARI_{method}_mean.pkl', 'wb') as f: 
            dump(total_ARI, f, HIGHEST_PROTOCOL)
    plot_experiment('mean', method)

In [ ]:
names = [g.graph['name'] for g in graphs]
# for method in methods:
#     plot_experiment(names, method)
plot_experiments(names, 'ward')

In [ ]:
def save_clustering(G, draw_dend=False):
    embedding = embed_snapshot_markov(G, 5, degrees=False)
    Z = cluster_walktrap(embedding, True)
    print(eval_ARI(G, Z))

    if draw_dend:
        draw_dendrogram(G, Z)
    root = to_tree(Z)
    with open(f'dendrogram_{G.graph["name"]}.pkl', 'wb') as f:
        dump((G,root), f, HIGHEST_PROTOCOL)

In [ ]:
save_clustering(wrangle_karate(), 'karate')
save_clustering(wrangle_football(), 'football')
save_clustering(wrangle_caltech(), 'caltech')
save_clustering(generate_lfr(), 'lfr')

# misc

In [ ]:
# try infomap

from infomap import Infomap
im = Infomap()
G = wrangle_caltech()
for i,j in G.edges:
    im.add_link(i,j)
im.run("--two-level --num-trials 5")
predicted = []
for node in im.tree:
    if node.is_leaf:
        predicted.append(node.module_id)
#         print(node.node_id, node.module_id)

gtruth = list(nx.get_node_attributes(G, 'cluster').values())
print(adjusted_rand_score(gtruth, predicted))

In [ ]:
# try pure modularity

from community import best_partition
G = wrangle_caltech()
predicted = list(best_partition(G).values())

gtruth = list(nx.get_node_attributes(G, 'cluster').values())
print(adjusted_rand_score(gtruth, predicted))

In [ ]:
# lesmis figure

G = wrangle_lesmis()

embedding = embed_snapshot_markov(G, 5, degrees=False)
Z = cluster_walktrap(embedding, True)
# Z = girvan_newman(G)

# TODO: save names of characters to place around circle!
draw_dendrogram(G, Z, (8,6))
plt.savefig('lesmis_dendrogram.png')
root = to_tree(Z)
with open(f'dendrogram_lesmis.pkl', 'wb') as f:
    dump((G,root), f, HIGHEST_PROTOCOL)